In [ ]:
# Model Training:
# 1) Load all data from preprocessing (training/test splits, etc)
# 2) Begin Training Models
    #  a) Decision Tree
    #  b) Naive Bayes
    #  c) Logistic Regression
    #  d) SVM
# 3) Testing Models
# 4) New Iterations

In [207]:
# Imports
from Models import ModelUtil
import importlib
from Data import Preprocessing
from sklearn.model_selection import train_test_split

In [301]:
# 1) Load all data from preprocessing 
importlib.reload(Preprocessing)
importlib.reload(ModelUtil)
infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering()

In [294]:
# 2) Training Models
Y = infieldDataFrame["FieldSlice"]
X = infieldDataFrame.drop(["TaggedHitType","PitchCall","FieldSlice","Direction"], axis=1)
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.25, random_state=11)

In [302]:
# a) Decision Tree
# Need to test these hyperparameters for best case
result = ModelUtil.runDT(xTrain, yTrain, xTest, yTest, 25, 10, 55)

# I could join the dfs for x, y, and predictions and then visualize them.
# Probably could start the filtering on Pitch types with Pitch Style(Left/Right) to just
# allow for some easy visualizations



print(result)

training decision tree model...
done!
getting statistics...

Decisions Tree Data Splits: train=0.75, test=0.25
Decision Tree Depth: 11
Decision Tree Number of Leaves: 55
Decision Tree's Accuracy Score for Predicting on Training Data: 0.7000
Decision Tree's Accuracy Score for Predicting on Test Data: 0.2430

Decision Tree Overall Average Probabilities
-------------------------------------
Section 1: 21.58%
Section 2: 24.40%
Section 3: 24.56%
Section 4: 19.74%
Section 5: 9.73%
Decision Tree Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		62		51
2		90		110
3		74		83
4		65		60
5		29		16
Amount Correct: 224
Amount Incorrect: 96
Decision Tree Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		19		20
2		29		32
3		25		30
4		24		20
5		10		5
Amount Correct: 26
Amount Incorrect: 81
done!
(DecisionTreeClassifier(max_depth=25, max_features=10, max_leaf_nodes=55), 0.7, 

c:\Users\laure\Documents\Auburn\Senior Design\shifting_model\Models\ModelUtil.py:64: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  # dfTestStats = pd.DataFrame()
c:\Users\laure\Documents\Auburn\Senior Design\shifting_model\Models\ModelUtil.py:68: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dftemp = dftest[dftest["Correct"] == True]
c:\Users\laure\Documents\Auburn\Senior Design\shifting_model\Models\ModelUtil.py:70: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the 

In [ ]:
# a) Decision Tree
# Need to test these hyperparameters for best case
result = ModelUtil.runDT(xTrain, yTrain, xTest, yTest, 25, 10, 55)

# I could join the dfs for x, y, and predictions and then visualize them.
# Probably could start the filtering on Pitch types with Pitch Style(Left/Right) to just
# allow for some easy visualizations



print(result)

training decision tree model...
done!
getting statistics...

Decisions Tree Data Splits: train=0.75, test=0.25
Decision Tree Depth: 11
Decision Tree Number of Leaves: 55
Decision Tree's Accuracy Score for Predicting on Training Data: 0.7000
Decision Tree's Accuracy Score for Predicting on Test Data: 0.2430

Decision Tree Overall Average Probabilities
-------------------------------------
Section 1: 21.58%
Section 2: 24.40%
Section 3: 24.56%
Section 4: 19.74%
Section 5: 9.73%
Decision Tree Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		62		51
2		90		110
3		74		83
4		65		60
5		29		16
Amount Correct: 224
Amount Incorrect: 96
Decision Tree Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		19		20
2		29		32
3		25		30
4		24		20
5		10		5
Amount Correct: 26
Amount Incorrect: 81
done!
(DecisionTreeClassifier(max_depth=25, max_features=10, max_leaf_nodes=55), 0.7, 

c:\Users\laure\Documents\Auburn\Senior Design\shifting_model\Models\ModelUtil.py:64: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  # dfTestStats = pd.DataFrame()
c:\Users\laure\Documents\Auburn\Senior Design\shifting_model\Models\ModelUtil.py:68: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dftemp = dftest[dftest["Correct"] == True]
c:\Users\laure\Documents\Auburn\Senior Design\shifting_model\Models\ModelUtil.py:70: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the 

In [ ]:
# b) Naive Bayes
result = ModelUtil.runNB()

In [ ]:
# c)Logistic Regression
result = ModelUtil.runLogReg()

In [ ]:
# d) SVM
result = ModelUtil.runSVM()

In [ ]:
# 3) Model Testing:

In [ ]:
# 4) Model Iterations and Improvements